In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
import copy
sys.path.append('./../')
from gan.models.small_models import Generator
from gan.config import SmallModelConfig
from gan.utils import check_playable_zelda,tensor_to_level_str
from NeuroEvolution import NeuroEvolution
import torch

config = SmallModelConfig()
config.set_env()
generator = Generator(config.input_shape[0],config.model_shapes,z_shape=(config.latent_size,),filters=config.generator_filters,use_linear4z2features_g=config.use_linear4z2features_g,use_self_attention=config.use_self_attention_g,use_deconv_g=config.use_deconv_g)

In [3]:
model_dict = torch.load(
    "/root/mnt/pcg/GVGAI-GAN/gan/checkpoints/none-311/models_3500.tar")
generator.load_state_dict(model_dict['generator'])


<All keys matched successfully>

In [4]:
for param in generator.parameters():
    print(param.shape)

torch.Size([32, 512, 3, 4])
torch.Size([512])
torch.Size([512])
torch.Size([256, 512, 3, 3])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([1])
torch.Size([32, 256, 1, 1])
torch.Size([32])
torch.Size([32, 256, 1, 1])
torch.Size([32])
torch.Size([256, 256, 1, 1])
torch.Size([256])
torch.Size([128, 256, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([1])
torch.Size([16, 128, 1, 1])
torch.Size([16])
torch.Size([16, 128, 1, 1])
torch.Size([16])
torch.Size([128, 128, 1, 1])
torch.Size([128])
torch.Size([8, 128, 1, 1])
torch.Size([8])


In [5]:


from functools import partial
import time


def get_reward(weights,model,render=False):
    cloned_generator = copy.deepcopy(generator)
    for i,param in enumerate(list(cloned_generator.parameters())[-8:]):
        try:
            param.data = weights[i]
        except:
            param.data = weights[i].data
    latents = torch.randn(100,32,)
    def generate_levels(latents, labels):
        p_level = torch.softmax(
            generator(latents, labels), dim=1)
        level_strs = tensor_to_level_str(
            'zelda', p_level)
        return level_strs
    levels = generate_levels(latents, labels=None)
    playable_levels = []
    for level_str in levels:
        if check_playable_zelda(level_str, 'v1'):
            playable_levels.append(level_str)
    del cloned_generator
    return len(playable_levels)

partial_func = partial(get_reward,model=generator)
mother_parameters =list(generator.parameters())[-8:]

ne = NeuroEvolution(
    mother_parameters,partial_func,population_size=15,sigma=0.1
)

start = time.time()
final_weights = ne.run(10000, print_step=1)
end = time.time() - start
print("Total time: {}", end)


KeyboardInterrupt: 